# Geochemical Data - Importing, Processing and 'Munging'

In this notebook we'll go through some of the tasks which `pyrolite` can help make a bit easier with respect to getting your data analysis ready.


## Importing Data

`pyrolite` is largely based around `pandas`, and as such you're typically working with Pandas dataframes. Pandas can work with [a variety of file types](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html), some more performant than others, but is also happy to digest humble CSV and Excel files (with the functions `pandas.read_csv` and `pandas.read_excel`, respectively). Some of these functions are able to directly connect to remote files (e.g. CSV) or directly to database connections. You'll see one or two examples of fetching a remote CSV file directly below and in other notebooks.

## Cleaning Up Column Names

One of the challenges of working with larger datasets is being able to quickly find the right data when you need it. `pyrolite` provides some functions for this, but for the time being is dependent on being able to recognise compositional columns by looking for elements, oxide names and isotopes (without unit annotations, delimiters and other markup). Here we show some of the steps which might be required to get your dataframe in a standardised format (using examples [from GEOROC](http://georoc.mpch-mainz.gwdg.de/georoc/Entry.html)). Notably, this can be the most difficut step of any analysis workflow, so being able to do this in a way which is repeatable might save you a decent amount of time if you have to do it again in the future!

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('http://georoc.mpch-mainz.gwdg.de/georoc/Csv_Downloads/Continental_Flood_Basalts_comp/CENTRAL_ATLANTIC_MAGMATIC_PROVINCE_-_CAMP.csv',
                 encoding='cp1252',
                 skip_blank_lines=False) # get some data from GEOROC directly
df = df.loc[:np.argmax(df.iloc[:, 0].isnull())-1] # omit the abbreviations and references after the blank line in this file

Quickly looking at this dataframe, we can see that all the column names are capitalised, `_` is used as a delimiter for isotopes, and units are given for the geochemical parameters in the format `(UNIT)`. We also have a unique index, and ean extra redundant column on the right hand side of our table (which we can drop):

In [2]:
df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,RE187_OS188,HF176_HF177,HE3_HE4,HE3_HE4(R/R(A)),HE4_HE3,HE4_HE3(R/R(A)),K40_AR40,AR40_K40,UNIQUE_ID,Unnamed: 171
0,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015617,NaN
1,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015618,NaN


In [3]:
df.drop(columns=[c for c in df.columns if 'Unnamed' in c], inplace=True) # drop our redundant column

We can alter the names of columns which `pyrolite` will recognise so we can use some of its more automated methods. Specifically, pyrolite expects element and oxide names to have 'title case' names (e.g. `Si`, `SiO2`, not `MG`, `MGO`). Similarly, for isotope ratios, it expects something along the lines of `87Sr/86Sr`. Here I've written a function which will attenpt to find relevant element and oxide names among the capitalised versions we find here.

In [4]:
from pyrolite.geochem.ind import __common_elements__, __common_oxides__ # indexes of elements and oxides which we'll check against

def rename_columns(df):
    """
    Rename the columns which pyrolite can access so we can use the indexing and transformation functions.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns you'd like to rename.
    
    Returns
    -------
    pandas.DataFrame
        Dataframe with columns renamed.
    """
    _elements, _oxides = {e.upper(): e for e in __common_elements__}, {o.upper(): o for o in __common_oxides__} # these will serve as lookup tables for our capitalised components
    
    element_columns = {c: _elements[c[:c.find("(PPM")]] for c in df.columns if c[:c.find("(PPM")] in _elements} # all of the elemental values are in ppm here
    oxide_columns = {c: _oxides[c[:c.find("(WT%")]] for c in df.columns if c[:c.find("(WT%")] in _oxides} # this omits some gas measurements, but gets the major oxides
    isotope_columns = {c: '/'.join([c.title() for c in c.split('_')]) for c in df.pyrochem.list_isotope_ratios if len(c.split('_'))==2}
    
    return df.rename(columns = {**element_columns, **oxide_columns, **isotope_columns})
    

df = rename_columns(df)

In [5]:
df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,Re187/Os186,Re187/Os188,Hf176/Hf177,He3/He4,He3/He4(R/R(A)),He4/He3,He4/He3(R/R(A)),K40/Ar40,Ar40/K40,UNIQUE_ID
0,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015617
1,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015618


Finally, it might be a good idea to use the Unique ID as the index for our dataframe:

In [6]:
df = df.set_index('UNIQUE_ID', drop=True)

You could package this up into a function to fetch one of these GEOROC files:

In [7]:
def fetch_GEOROC_csv(filepath):
    """
    Fetch a GEOROC csv from a local file or URL.
    
    Parameters
    ----------
    filepath : str | pathlib.Path
        Filepath to a GEOROC csv - can be used to directly fetch a URL thanks to `pandas.read_csv`.
    
    Returns
    -------
    pandas.DataFrame
        Dataframe formatted for `pyrolite`.
    """
    df = pd.read_csv(filepath, encoding='cp1252', skip_blank_lines=False) # get some data from GEOROC directly
    df = df.loc[:np.argmax(df.iloc[:, 0].isnull())-1] # omit the abbreviations and references after the blank line in this file
    df.drop(columns=[c for c in df.columns if 'Unnamed' in c], inplace=True) # drop our redundant column
    df = rename_columns(df)
    df = df.set_index('UNIQUE_ID', drop=True)
    return df

Now we can use this to fetch another CSV - this time from the Kermadec Arc. Note that the column names which `pyrolite` can work with are already converted to usable versions:

In [8]:
kermadec_df = fetch_GEOROC_csv('http://georoc.mpch-mainz.gwdg.de/georoc/Csv_Downloads/Convergent_Margins_comp/KERMADEC_ARC.csv')
kermadec_df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,Os187/Os188,Re187/Os186,Re187/Os188,Hf176/Hf177,He3/He4,He3/He4(R/R(A)),He4/He3,He4/He3(R/R(A)),K40/Ar40,Ar40/K40
UNIQUE_ID,,,,,,,,,,,,,,,,,,,,,
10203-46325,[13460][9592],CONVERGENT MARGIN,KERMADEC ARC / KERMADEC ISLANDS / RAOUL / RAOUL,NaN,-29.25,-29.25,-177.87,-177.87,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10203-46329,[13460][9592],CONVERGENT MARGIN,KERMADEC ARC / KERMADEC ISLANDS / RAOUL / RAOUL,NaN,-29.25,-29.25,-177.87,-177.87,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Selecting Subsets of your Data

The `pyrolite.pyrochem` API provides access to indexing and transformation functions. This allows easy subsetting of geochemical datasets which can otherwise be unweildly (expecially as the number of columns increases..). To provide a simple illustration we generate a synthetic dataset to work from, which contains an array of typical geochemical measures - oxide components, element components (here as ppm), element ratios and isotope ratios. While this size dataset is managable, some of the indexing tools pyrolite provides make it straightforward to pull out different parts of the dataset.

In [9]:
import numpy as np
from pyrolite.util.synthetic import normal_frame

df = normal_frame(columns=['CaO', 'MgO', 'SiO2', 'FeO','Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te']) * 100
df[['Ni', 'Ti', 'La', 'Lu', 'Te']] *= 10
df.pyrochem.add_ratio('Mg/Fe') # one way to add an element ratio to a dataframe!
df['Sr87/Sr86'] = 0.0700  / 0.0986 + np.random.randn(df.index.size) * 0.0001
df

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Mg/Fe,Sr87/Sr86
0,16.758453,9.093743,2.817138,27.895706,2.331119,79.286684,13.598710,74.138300,191.972896,52.041815,0.252905,0.709795
1,15.691375,9.347256,3.190244,26.458924,2.681551,81.523819,14.150393,72.927694,205.101368,52.603219,0.274071,0.710120
2,16.469040,9.261111,2.995909,26.183172,2.367468,79.757968,12.426286,73.212968,207.581576,54.254201,0.274405,0.709931
3,15.798516,9.312314,3.032703,27.282302,2.511405,79.954316,13.467592,72.602572,203.872093,50.731024,0.264806,0.709795
4,16.895475,9.180245,2.899210,27.738101,2.369946,79.246921,13.093186,72.439711,192.688374,51.702034,0.256761,0.709897
5,15.125135,9.312320,3.281935,24.828905,2.805410,80.510462,13.328165,72.088863,227.249060,53.286386,0.290973,0.709989
6,15.899598,9.558891,3.324023,23.286646,2.457145,80.382090,11.310539,70.954117,235.455806,56.634410,0.318458,0.710154
7,15.149679,9.601653,3.521781,24.566315,2.754511,82.683525,12.366782,71.447077,223.291558,54.271665,0.303220,0.710012
8,15.588923,9.223201,3.050066,26.210781,2.589866,79.866032,13.755861,74.147120,213.925860,51.676761,0.272994,0.709995
9,16.102695,9.362847,2.957771,27.400750,2.310744,79.841108,12.808779,74.901498,199.750875,51.349682,0.265092,0.710021


In [10]:
df.pyrochem.oxides

,CaO,MgO,SiO2,FeO,Na2O
0,16.758453,9.093743,2.817138,27.895706,2.331119
1,15.691375,9.347256,3.190244,26.458924,2.681551
2,16.469040,9.261111,2.995909,26.183172,2.367468
3,15.798516,9.312314,3.032703,27.282302,2.511405
4,16.895475,9.180245,2.899210,27.738101,2.369946
5,15.125135,9.312320,3.281935,24.828905,2.805410
6,15.899598,9.558891,3.324023,23.286646,2.457145
7,15.149679,9.601653,3.521781,24.566315,2.754511
8,15.588923,9.223201,3.050066,26.210781,2.589866
9,16.102695,9.362847,2.957771,27.400750,2.310744


In [11]:
df.pyrochem.elements

,Ni,Ti,La,Lu,Te
0,79.286684,13.598710,74.138300,191.972896,52.041815
1,81.523819,14.150393,72.927694,205.101368,52.603219
2,79.757968,12.426286,73.212968,207.581576,54.254201
3,79.954316,13.467592,72.602572,203.872093,50.731024
4,79.246921,13.093186,72.439711,192.688374,51.702034
5,80.510462,13.328165,72.088863,227.249060,53.286386
6,80.382090,11.310539,70.954117,235.455806,56.634410
7,82.683525,12.366782,71.447077,223.291558,54.271665
8,79.866032,13.755861,74.147120,213.925860,51.676761
9,79.841108,12.808779,74.901498,199.750875,51.349682


In [12]:
df.pyrochem.REE

,La,Lu
0,74.138300,191.972896
1,72.927694,205.101368
2,73.212968,207.581576
3,72.602572,203.872093
4,72.439711,192.688374
5,72.088863,227.249060
6,70.954117,235.455806
7,71.447077,223.291558
8,74.147120,213.925860
9,74.901498,199.750875


In [13]:
df.pyrochem.compositional

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te
0,16.758453,9.093743,2.817138,27.895706,2.331119,79.286684,13.598710,74.138300,191.972896,52.041815
1,15.691375,9.347256,3.190244,26.458924,2.681551,81.523819,14.150393,72.927694,205.101368,52.603219
2,16.469040,9.261111,2.995909,26.183172,2.367468,79.757968,12.426286,73.212968,207.581576,54.254201
3,15.798516,9.312314,3.032703,27.282302,2.511405,79.954316,13.467592,72.602572,203.872093,50.731024
4,16.895475,9.180245,2.899210,27.738101,2.369946,79.246921,13.093186,72.439711,192.688374,51.702034
5,15.125135,9.312320,3.281935,24.828905,2.805410,80.510462,13.328165,72.088863,227.249060,53.286386
6,15.899598,9.558891,3.324023,23.286646,2.457145,80.382090,11.310539,70.954117,235.455806,56.634410
7,15.149679,9.601653,3.521781,24.566315,2.754511,82.683525,12.366782,71.447077,223.291558,54.271665
8,15.588923,9.223201,3.050066,26.210781,2.589866,79.866032,13.755861,74.147120,213.925860,51.676761
9,16.102695,9.362847,2.957771,27.400750,2.310744,79.841108,12.808779,74.901498,199.750875,51.349682


In [14]:
df.pyrochem.isotope_ratios

,Sr87/Sr86
0,0.709795
1,0.710120
2,0.709931
3,0.709795
4,0.709897
5,0.709989
6,0.710154
7,0.710012
8,0.709995
9,0.710021


Notably, these dataframe accessors can also be used to re-assign values back to the dataframe. Here we transform element components to wt% equivalents by dividing by 10000, and note that the change has been incorporated into our dataframe:

In [15]:
df.pyrochem.elements /= 10000
df.pyrochem.elements

,Ni,Ti,La,Lu,Te
0,0.007929,0.001360,0.007414,0.019197,0.005204
1,0.008152,0.001415,0.007293,0.020510,0.005260
2,0.007976,0.001243,0.007321,0.020758,0.005425
3,0.007995,0.001347,0.007260,0.020387,0.005073
4,0.007925,0.001309,0.007244,0.019269,0.005170
5,0.008051,0.001333,0.007209,0.022725,0.005329
6,0.008038,0.001131,0.007095,0.023546,0.005663
7,0.008268,0.001237,0.007145,0.022329,0.005427
8,0.007987,0.001376,0.007415,0.021393,0.005168
9,0.007984,0.001281,0.007490,0.019975,0.005135


If you're just after a list of the relevant column names, there are respective functions for that too:

In [16]:
df.pyrochem.list_oxides

['CaO', 'MgO', 'SiO2', 'FeO', 'Na2O']

## Unit Scales

While you can transform elements and oxide abundnace units easily when you remember the relative scales, `pyrolite` provides some functions such that you don't have to rely on your memory. Here we create a copy of the dataframe and within it revert the change we made above - so these should be the orignal ppm values. This method provides an easy way to explicitly declare your intention when changing units - and makes sure the relative scales are correct!

In [17]:
df.pyrochem.elements.pyrochem.scale('wt%', 'ppm') # wt% to ppm

,Ni,Ti,La,Lu,Te
0,79.286684,13.598710,74.138300,191.972896,52.041815
1,81.523819,14.150393,72.927694,205.101368,52.603219
2,79.757968,12.426286,73.212968,207.581576,54.254201
3,79.954316,13.467592,72.602572,203.872093,50.731024
4,79.246921,13.093186,72.439711,192.688374,51.702034
5,80.510462,13.328165,72.088863,227.249060,53.286386
6,80.382090,11.310539,70.954117,235.455806,56.634410
7,82.683525,12.366782,71.447077,223.291558,54.271665
8,79.866032,13.755861,74.147120,213.925860,51.676761
9,79.841108,12.808779,74.901498,199.750875,51.349682


## Converting Chemical Components 

`pyrolite` provides some straightfoward methods to calcuate element-oxide conversions (e.g. to transform Ti abundance to TiO2 abudnance), assuming that the system is open to oxygen (i.e. in this case the extra oxygen will be added to the composition). This interface also allows the user to quickly add ratios and specify redox pairs at the same time. For example, we can transform a copy of our dataframe to include extra ratios and change some of our oxide components to elements:

In [18]:
df.pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "FeO", "Ca", "Te", "Na", "Na/Te", "MgO/SiO2"]
)

,Mg/Fe,Sr87/Sr86,MgO,SiO2,FeO,Ca,Te,Na,Na/Te,MgO/SiO2
0,0.252905,0.709795,9.093743,2.817138,27.895706,11.977112,0.005204,1.729358,332.301659,3.228007
1,0.274071,0.710120,9.347256,3.190244,26.458924,11.214481,0.005260,1.989329,378.176255,2.929950
2,0.274405,0.709931,9.261111,2.995909,26.183172,11.770271,0.005425,1.756324,323.721229,3.091252
3,0.264806,0.709795,9.312314,3.032703,27.282302,11.291054,0.005073,1.863104,367.251485,3.070631
4,0.256761,0.709897,9.180245,2.899210,27.738101,12.075040,0.005170,1.758162,340.056687,3.166464
5,0.290973,0.709989,9.312320,3.281935,24.828905,10.809795,0.005329,2.081214,390.571479,2.837448
6,0.318458,0.710154,9.558891,3.324023,23.286646,11.363296,0.005663,1.822852,321.862900,2.875699
7,0.303220,0.710012,9.601653,3.521781,24.566315,10.827336,0.005427,2.043455,376.523315,2.726363
8,0.272994,0.709995,9.223201,3.050066,26.210781,11.141259,0.005168,1.921312,371.794132,3.023935
9,0.265092,0.710021,9.362847,2.957771,27.400750,11.508447,0.005135,1.714242,333.837008,3.165508


In a similar way, we can also specify the molar speciation for redox species (so far just iron; others could be incorporated if they'll be useful). Here we adjust the total iron within our compositions (currently specified as FeO) to have a $Fe^{2+}/Fe^{3+}$ ratio of 9:1 (roughly what you might expect from a normal mantle-derived magma):

In [19]:
df.pyrochem.convert_chemistry(to=[{"FeO": 0.9, "Fe2O3": 0.1}])

,Mg/Fe,Sr87/Sr86,FeO,Fe2O3
0,0.252905,0.709795,25.106136,3.100183
1,0.274071,0.710120,23.813032,2.940506
2,0.274405,0.709931,23.564855,2.909860
3,0.264806,0.709795,24.554072,3.032012
4,0.256761,0.709897,24.964291,3.082667
5,0.290973,0.709989,22.346015,2.759354
6,0.318458,0.710154,20.957981,2.587956
7,0.303220,0.710012,22.109683,2.730171
8,0.272994,0.709995,23.589703,2.912929
9,0.265092,0.710021,24.660675,3.045176
